# Тест пайплайна

In [1]:
import os
import torch
import torch.nn.functional as F
import torch.nn as nn
import matplotlib.pyplot as plt
from torch.optim import Adam
import time
from tqdm import tqdm

## Данные

In [4]:
from torchvision import datasets, transforms

if torch.cuda.device_count() > 0:
    gpu_available = True
else:
    gpu_available = False

transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.5,), (0.5,))
])

dataset1 = datasets.MNIST('/homes/flomakin/gun_crypto_system/data',
                train=True, download=True,
                transform=transform)

dataset2 = datasets.MNIST('/homes/flomakin/gun_crypto_system/data',
                train=False, download=True,
                transform=transform)

train_kwargs = {'batch_size': int(len(dataset1)/4)}
test_kwargs = {'batch_size': len(dataset2)}

train_loader = torch.utils.data.DataLoader(dataset1, drop_last=True, **train_kwargs)
test_loader = torch.utils.data.DataLoader(dataset2, drop_last=True, **test_kwargs)

In [5]:
i = 0
for batch_idx, (data, target) in enumerate(train_loader):
    i+=1
    if i == 6:
        break

In [5]:
data.shape

torch.Size([60000, 1, 28, 28])

## Модель 1 - Энкодер Декодер

In [5]:
conv1 = nn.Conv2d(1, 32, kernel_size=3, stride=2, padding=1)
conv2 = nn.Conv2d(32, 96, kernel_size=3, stride=2, padding=1)
conv3 = nn.Conv2d(96, 288, kernel_size=3, stride=2, padding=1)
conv4 = nn.Conv2d(288, 784, kernel_size=5, stride=2, padding=1)
deconv4 = nn.ConvTranspose2d(784, 288, kernel_size=5, stride=2, padding=1, output_padding=1)
deconv3 = nn.ConvTranspose2d(288, 96, kernel_size=3, stride=2, padding=1, output_padding=0)
deconv2 = nn.ConvTranspose2d(96, 32, kernel_size=3, stride=2, padding=1, output_padding=1)
deconv1 = nn.ConvTranspose2d(32, 1, kernel_size=3, stride=2, padding=1, output_padding=1)

In [6]:
# image = torch.randn(60000, 1, 28, 28)
image = data
image.shape

torch.Size([60000, 1, 28, 28])

In [7]:
key = torch.randn(60000, 1, 128)

In [8]:
key.shape

torch.Size([60000, 1, 128])

In [141]:
a=conv1(image)
print(a.shape)
b=conv2(a)
print(b.shape)
c=conv3(b)
print(c.shape)
d = conv4(c)
print(d.shape)
e = deconv4(d)
print(e.shape)
f = deconv3(e)
print(f.shape)
g = deconv2(f)
print(g.shape)
h = deconv1(g)
print(h.shape)

torch.Size([60000, 32, 14, 14])
torch.Size([60000, 96, 7, 7])
torch.Size([60000, 288, 4, 4])
torch.Size([60000, 784, 1, 1])
torch.Size([60000, 288, 4, 4])
torch.Size([60000, 96, 7, 7])
torch.Size([60000, 32, 14, 14])
torch.Size([60000, 1, 28, 28])


In [9]:
d.view(d.shape[0], 1, -1).shape

NameError: name 'd' is not defined

In [ ]:
d.view(d.shape[0], 1, -1).cuda().shape, k.shape

In [10]:
torch.cat([d.view(d.shape[0], 1, -1), key], dim=-1).shape

NameError: name 'd' is not defined

In [145]:
dod = d.view(d.shape[0], 1, 1, 784)

In [19]:
class Encoder(nn.Module):
    def __init__(self,):
        super(Encoder, self).__init__()
        
        # Вход [1, 28, 28]
        self.conv1 = nn.Conv2d(1, 32, kernel_size=3, stride=2, padding=1)
        self.conv2 = nn.Conv2d(32, 96, kernel_size=3, stride=2, padding=1)
        self.conv3 = nn.Conv2d(96, 288, kernel_size=3, stride=2, padding=1)
        self.conv4 = nn.Conv2d(288, 784, kernel_size=5, stride=2, padding=1)
        # Выход [784, 1, 1]
        
        self.fc1 = nn.Linear(784 + 128, 784)
        self.fc2 = nn.Linear(784, 784)
        self.fc3 = nn.Linear(784, 784)
        
        # PReLU activation
        self.prelu = nn.PReLU()
        
    def forward(self, x, k):
        # Проход через сверточные слои
        x = self.prelu(self.conv1(x))
        x = self.prelu(self.conv2(x))
        x = self.prelu(self.conv3(x))
        x = self.prelu(self.conv4(x))
        
        # Примешиваем ключ
        # x = torch.cat([x.view(x.shape[0], 1, 784), k], dim=1)
        x = torch.cat([x.view(x.shape[0], 1, 784), k], dim=-1)
        
        x = self.prelu(self.fc1(x))
        x = self.prelu(self.fc2(x))
        x = nn.Tanh()(self.fc3(x)) 
        
        return x
    
class DecoderBOB(nn.Module):
    def __init__(self):
        super(DecoderBOB, self).__init__()

        self.fc4 = nn.Linear(784 + 128, 784)
        self.fc5 = nn.Linear(784, 784)
        self.fc6 = nn.Linear(784, 784)

        self.deconv4 = nn.ConvTranspose2d(784, 288, kernel_size=5, stride=2, padding=1, output_padding=1)
        self.deconv3 = nn.ConvTranspose2d(288, 96, kernel_size=3, stride=2, padding=1, output_padding=0)
        self.deconv2 = nn.ConvTranspose2d(96, 32, kernel_size=3, stride=2, padding=1, output_padding=1)
        self.deconv1 = nn.ConvTranspose2d(32, 1, kernel_size=3, stride=2, padding=1, output_padding=1)
        self.prelu = nn.PReLU()

    def forward(self, x, k):
        x = self.prelu(self.fc4(torch.cat([x, k], dim=-1)))
        x = self.prelu(self.fc5(x))
        x = self.prelu(self.fc6(x))

        x = x.view(x.shape[0], 784, 1, 1)

        x = self.prelu(self.deconv4(x))
        x = self.prelu(self.deconv3(x))
        x = self.prelu(self.deconv2(x))
        x = nn.Tanh()(self.deconv1(x)) 

        return x

class DecoderEVA(nn.Module):
    def __init__(self):
        super(DecoderEVA, self).__init__()

        self.fc4 = nn.Linear(784, 784)
        self.fc5 = nn.Linear(784, 784)
        self.fc6 = nn.Linear(784, 784)

        self.deconv4 = nn.ConvTranspose2d(784, 288, kernel_size=5, stride=2, padding=1, output_padding=1)
        self.deconv3 = nn.ConvTranspose2d(288, 96, kernel_size=3, stride=2, padding=1, output_padding=0)
        self.deconv2 = nn.ConvTranspose2d(96, 32, kernel_size=3, stride=2, padding=1, output_padding=1)
        self.deconv1 = nn.ConvTranspose2d(32, 1, kernel_size=3, stride=2, padding=1, output_padding=1)

        self.prelu = nn.PReLU()

    def forward(self, x):
        x = self.prelu(self.fc4(x))
        x = self.prelu(self.fc5(x))
        x = self.prelu(self.fc6(x))

        x = x.view(x.shape[0], 784, 1, 1)

        x = self.prelu(self.deconv4(x))
        x = self.prelu(self.deconv3(x))
        x = self.prelu(self.deconv2(x))
        x = nn.Tanh()(self.deconv1(x))  

        return x

In [13]:
en = Encoder()
debob = DecoderBOB()
deeva = DecoderEVA()

In [14]:
image = data
key = torch.randn(data.shape[0], 1, 128)
encoded_output = en(image, key)
encoded_output.shape

torch.Size([15000, 1, 784])

In [15]:
eva = deeva(encoded_output)
eva.shape

torch.Size([15000, 1, 28, 28])

In [16]:
bob = debob(encoded_output, key)
bob.shape

torch.Size([15000, 1, 28, 28])

## ЭКСП

In [22]:
training_steps = 1000
learning_rate = 10e-3
clip_value = 1
def generate_key_batch(size: int = 128, batchsize: int = 1, gpu_available: bool = True):
    if gpu_available:
        return torch.randn(batchsize, 1, size).cuda()
    else:
        return torch.randn(batchsize, 1, size)

In [23]:
alice = Encoder()
bob = DecoderBOB()
eve = DecoderEVA()

alice.train()
bob.train()
eve.train()

if gpu_available:
    alice.cuda()
    bob.cuda()
    eve.cuda()

aggregated_losses = {
        "alice_bob_training_loss": [],
        "bob_reconstruction_training_errors": [],
        "eve_reconstruction_training_errors": [],
        "step": []
}

optimizer_alice = Adam(params=alice.parameters(), lr=learning_rate)
optimizer_bob = Adam(params=bob.parameters(), lr=learning_rate)
optimizer_eve = Adam(params=eve.parameters(), lr=learning_rate)

# define losses 
bob_reconstruction_error = nn.L1Loss()
eve_reconstruction_error = nn.L1Loss()

for batch_idx, (data, target) in tqdm(enumerate(train_loader)):
    data = data.to('cuda')
    # Training alternates between Alice/Bob and Eve
    for network, num_minibatches in {"alice_bob": 1, "eve": 2}.items():
        """ 
        Alice/Bob training for one minibatch, and then Eve training for two minibatches this ratio 
        in order to give a slight computational edge to the adversary Eve without training it so much
        that it becomes excessively specific to the exact current parameters of Alice and Bob
        """
        for _ in range(num_minibatches):

            k = generate_key_batch(size=128, batchsize=data.shape[0], gpu_available=gpu_available)

            # forward pass through alice and eve networks

            ciphertext = alice.forward(data, k)

            eve_p = eve.forward(ciphertext)

            if network == "alice_bob":

                # forward pass through bob network
                bob_p = bob.forward(ciphertext, k)

                # calculate errors
                error_bob = bob_reconstruction_error(input=bob_p, target=data)
                error_eve = eve_reconstruction_error(input=eve_p, target=data)
                alice_bob_loss =  error_bob + F.relu(1 - error_eve)

                # Zero gradients, perform a backward pass, clip gradients, and update the weights.
                optimizer_alice.zero_grad()
                optimizer_bob.zero_grad()
                alice_bob_loss.backward()
                nn.utils.clip_grad_value_(alice.parameters(), clip_value)
                nn.utils.clip_grad_value_(bob.parameters(), clip_value)
                optimizer_alice.step()
                optimizer_bob.step()

            elif network == "eve":

                # calculate error
                error_eve = eve_reconstruction_error(input=eve_p, target=data)

                # Zero gradients, perform a backward pass, and update the weights
                optimizer_eve.zero_grad()
                error_eve.backward()
                nn.utils.clip_grad_value_(eve.parameters(), clip_value)
                optimizer_eve.step()

4it [03:17, 49.43s/it]
